# 🚀 Déploiement du Modèle SSD MobileNet V2

Ce notebook automatise le déploiement du modèle SSD MobileNet V2 entraîné vers le système de versioning de l'API.

## 📋 Processus de déploiement

1. **Analyse du modèle** : Vérification du modèle SavedModel
2. **Versioning automatique** : Incrémentation intelligente de la version
3. **Copie et métadonnées** : Déploiement avec informations complètes
4. **Validation** : Test du modèle déployé

In [1]:
import os
import shutil
import json
import tensorflow as tf
from datetime import datetime
from pathlib import Path
import re

# Configuration des chemins
TRAINING_MODEL_PATH = "/home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/exported_model/saved_model"
API_VERSIONS_PATH = "/home/sarsator/projets/gaia_vision/api/models/dl_model/versions"
TRAINING_NOTEBOOK_PATH = "/home/sarsator/projets/gaia_vision/training/notebook/dl_finetuning.ipynb"

print("🔧 Configuration du déploiement")
print(f"📁 Modèle source: {TRAINING_MODEL_PATH}")
print(f"📁 Destination API: {API_VERSIONS_PATH}")
print(f"📁 Notebook source: {TRAINING_NOTEBOOK_PATH}")

# Vérification des chemins
assert os.path.exists(TRAINING_MODEL_PATH), f"❌ Modèle source non trouvé: {TRAINING_MODEL_PATH}"
assert os.path.exists(API_VERSIONS_PATH), f"❌ Dossier de versions non trouvé: {API_VERSIONS_PATH}"
assert os.path.exists(TRAINING_NOTEBOOK_PATH), f"❌ Notebook source non trouvé: {TRAINING_NOTEBOOK_PATH}"

print("✅ Tous les chemins sont valides!")

2025-07-16 13:25:11.048155: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-16 13:25:11.072481: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-16 13:25:11.072515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-16 13:25:11.073274: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-16 13:25:11.078016: I tensorflow/core/platform/cpu_feature_guar

🔧 Configuration du déploiement
📁 Modèle source: /home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/exported_model/saved_model
📁 Destination API: /home/sarsator/projets/gaia_vision/api/models/dl_model/versions
📁 Notebook source: /home/sarsator/projets/gaia_vision/training/notebook/dl_finetuning.ipynb
✅ Tous les chemins sont valides!


In [2]:
def get_next_version():
    """Analyse les versions existantes et retourne la prochaine version à utiliser"""
    
    if not os.path.exists(API_VERSIONS_PATH):
        return "1.0", "20250716_000000"
    
    # Lister toutes les versions existantes
    versions = []
    for folder in os.listdir(API_VERSIONS_PATH):
        if os.path.isdir(os.path.join(API_VERSIONS_PATH, folder)):
            # Format attendu: v1.2_20250711_143350
            match = re.match(r'v(\d+)\.(\d+)_(\d{8}_\d{6})', folder)
            if match:
                major, minor, timestamp = match.groups()
                versions.append((int(major), int(minor), timestamp, folder))
    
    if not versions:
        return "1.0", "20250716_000000"
    
    # Trier par version (major, minor)
    versions.sort(key=lambda x: (x[0], x[1]))
    latest_major, latest_minor, _, latest_folder = versions[-1]
    
    print(f"📋 Versions existantes trouvées:")
    for major, minor, timestamp, folder in versions:
        print(f"   └── v{major}.{minor}_{timestamp}")
    
    print(f"🔍 Dernière version: v{latest_major}.{latest_minor}")
    
    # Incrémenter la version mineure
    next_version = f"{latest_major}.{latest_minor + 1}"
    next_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print(f"🆕 Nouvelle version: v{next_version}_{next_timestamp}")
    
    return next_version, next_timestamp

# Test de la fonction
next_version, next_timestamp = get_next_version()

📋 Versions existantes trouvées:
   └── v1.0_20250711_143245
   └── v1.1_20250711_143319
   └── v1.2_20250711_143350
🔍 Dernière version: v1.2
🆕 Nouvelle version: v1.3_20250716_132518


In [3]:
def analyze_savedmodel():
    """Analyse le modèle SavedModel et retourne ses informations"""
    
    print("🔍 Analyse du modèle SavedModel...")
    
    # Charger le modèle
    try:
        model = tf.saved_model.load(TRAINING_MODEL_PATH)
        print("✅ Modèle chargé avec succès")
    except Exception as e:
        print(f"❌ Erreur lors du chargement: {e}")
        return None
    
    # Analyser les signatures
    signatures = list(model.signatures.keys())
    print(f"📋 Signatures disponibles: {signatures}")
    
    # Analyser la signature principale (généralement 'serving_default')
    if 'serving_default' in signatures:
        serving_fn = model.signatures['serving_default']
        
        print("\n🔧 Analyse de la signature 'serving_default':")
        print("   📥 Inputs:")
        for name, spec in serving_fn.structured_input_signature[1].items():
            print(f"      └── {name}: {spec.shape} ({spec.dtype})")
        
        print("   📤 Outputs:")
        for name, spec in serving_fn.structured_outputs.items():
            print(f"      └── {name}: {spec.shape} ({spec.dtype})")
    
    # Calculer la taille du modèle
    total_size = 0
    for root, dirs, files in os.walk(TRAINING_MODEL_PATH):
        for file in files:
            total_size += os.path.getsize(os.path.join(root, file))
    
    size_mb = total_size / (1024 * 1024)
    print(f"\n📏 Taille du modèle: {size_mb:.2f} MB ({total_size} bytes)")
    
    model_info = {
        'signatures': signatures,
        'size_bytes': total_size,
        'size_mb': round(size_mb, 2),
        'architecture': 'SSD_MobileNet_V2_320x320',
        'framework': 'TensorFlow_Object_Detection_API'
    }
    
    return model_info

# Analyser le modèle
model_info = analyze_savedmodel()

🔍 Analyse du modèle SavedModel...


2025-07-16 13:25:22.451643: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-16 13:25:22.521034: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-16 13:25:22.521069: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-16 13:25:22.522598: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:02:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-16 13:25:22.522624: I external/local_xla/xla/stream_executor

✅ Modèle chargé avec succès
📋 Signatures disponibles: ['serving_default']

🔧 Analyse de la signature 'serving_default':
   📥 Inputs:
      └── input_tensor: (1, None, None, 3) (<dtype: 'uint8'>)
   📤 Outputs:
      └── raw_detection_boxes: (1, 2034, 4) (<dtype: 'float32'>)
      └── detection_multiclass_scores: (1, 100, 3) (<dtype: 'float32'>)
      └── detection_classes: (1, 100) (<dtype: 'float32'>)
      └── detection_boxes: (1, 100, 4) (<dtype: 'float32'>)
      └── raw_detection_scores: (1, 2034, 3) (<dtype: 'float32'>)
      └── num_detections: (1,) (<dtype: 'float32'>)
      └── detection_anchor_indices: (1, 100) (<dtype: 'float32'>)
      └── detection_scores: (1, 100) (<dtype: 'float32'>)

📏 Taille du modèle: 22.83 MB (23936569 bytes)


In [5]:
def deploy_model(version, timestamp, model_info):
    """Déploie le modèle avec versioning automatique"""
    
    # Créer le nom du dossier de destination
    version_folder = f"v{version}_{timestamp}"
    destination_path = os.path.join(API_VERSIONS_PATH, version_folder)
    
    print(f"🚀 Déploiement vers: {destination_path}")
    
    # Créer le dossier de destination
    os.makedirs(destination_path, exist_ok=True)
    
    # Copier le modèle SavedModel
    model_dest_path = os.path.join(destination_path, "saved_model")
    print(f"📁 Copie du modèle SavedModel...")
    
    if os.path.exists(model_dest_path):
        shutil.rmtree(model_dest_path)
    
    shutil.copytree(TRAINING_MODEL_PATH, model_dest_path)
    print(f"✅ Modèle copié vers: {model_dest_path}")
    
    # Créer les métadonnées
    metadata = {
        "version": version,
        "timestamp": datetime.now().isoformat(),
        "model_type": "deep_learning_object_detection",
        "deployed_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "deployment_id": timestamp,
        "model_size_mb": model_info['size_mb'],
        "model_size_bytes": model_info['size_bytes'],
        "architecture": model_info['architecture'],
        "framework": model_info['framework'],
        "signatures": model_info['signatures'],
        "deployed_by": "automated_deployment_notebook",
        "training_notebook": "dl_finetuning.ipynb",
        "source_path": TRAINING_MODEL_PATH,
        "deployment_type": "production_savedmodel",
        "model_format": "tensorflow_savedmodel",
        "deployed_path": model_dest_path,
        "performance_metrics": {
            "precision": 99.8,
            "recall": 96.0,
            "f1_score": 97.9,
            "inference_time_ms": 16,
            "test_images": 394,
            "training_steps": 30000
        },
        "model_capabilities": [
            "mushroom_detection",
            "contamination_classification", 
            "real_time_inference",
            "batch_processing"
        ],
        "input_specifications": {
            "image_size": "320x320",
            "channels": 3,
            "format": "RGB",
            "normalization": "0-255"
        },
        "output_specifications": {
            "detection_boxes": "N boxes with coordinates",
            "detection_classes": "Class IDs (1: healthy, 2: contaminated)",
            "detection_scores": "Confidence scores 0-1",
            "num_detections": "Number of valid detections"
        }
    }
    
    # Sauvegarder les métadonnées
    metadata_path = os.path.join(destination_path, "metadata.json")
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Métadonnées sauvegardées: {metadata_path}")
    
    # Copier le label_map.pbtxt si disponible
    label_map_source = "/home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/label_map.pbtxt"
    if os.path.exists(label_map_source):
        label_map_dest = os.path.join(destination_path, "label_map.pbtxt")
        shutil.copy2(label_map_source, label_map_dest)
        print(f"✅ Label map copié: {label_map_dest}")
    
    return destination_path, metadata

print("🎯 Prêt pour le déploiement!")

🎯 Prêt pour le déploiement!


In [6]:
# 🚀 EXÉCUTION DU DÉPLOIEMENT
print("="*60)
print("🚀 DÉMARRAGE DU DÉPLOIEMENT AUTOMATIQUE")
print("="*60)

if model_info is not None:
    # Déployer le modèle
    deployed_path, metadata = deploy_model(next_version, next_timestamp, model_info)
    
    print("\n" + "="*60)
    print("✅ DÉPLOIEMENT TERMINÉ AVEC SUCCÈS!")
    print("="*60)
    print(f"📁 Modèle déployé dans: {deployed_path}")
    print(f"🏷️  Version: v{next_version}_{next_timestamp}")
    print(f"📏 Taille: {model_info['size_mb']} MB")
    print(f"🏗️  Architecture: {model_info['architecture']}")
    print(f"⚡ Performance: F1-Score 97.9% | Précision 99.8% | Rappel 96.0%")
    print(f"🕐 Temps d'inférence: 16ms par image")
    
else:
    print("❌ Échec de l'analyse du modèle - Déploiement annulé")

🚀 DÉMARRAGE DU DÉPLOIEMENT AUTOMATIQUE
🚀 Déploiement vers: /home/sarsator/projets/gaia_vision/api/models/dl_model/versions/v1.3_20250716_132518
📁 Copie du modèle SavedModel...
✅ Modèle copié vers: /home/sarsator/projets/gaia_vision/api/models/dl_model/versions/v1.3_20250716_132518/saved_model
✅ Métadonnées sauvegardées: /home/sarsator/projets/gaia_vision/api/models/dl_model/versions/v1.3_20250716_132518/metadata.json
✅ Label map copié: /home/sarsator/projets/gaia_vision/api/models/dl_model/versions/v1.3_20250716_132518/label_map.pbtxt

✅ DÉPLOIEMENT TERMINÉ AVEC SUCCÈS!
📁 Modèle déployé dans: /home/sarsator/projets/gaia_vision/api/models/dl_model/versions/v1.3_20250716_132518
🏷️  Version: v1.3_20250716_132518
📏 Taille: 22.83 MB
🏗️  Architecture: SSD_MobileNet_V2_320x320
⚡ Performance: F1-Score 97.9% | Précision 99.8% | Rappel 96.0%
🕐 Temps d'inférence: 16ms par image


In [7]:
def validate_deployment(deployed_path):
    """Valide que le déploiement s'est bien déroulé"""
    
    print("\n🔍 VALIDATION DU DÉPLOIEMENT")
    print("-" * 40)
    
    checks = []
    
    # Vérifier que le dossier existe
    if os.path.exists(deployed_path):
        checks.append("✅ Dossier de déploiement créé")
    else:
        checks.append("❌ Dossier de déploiement manquant")
        return False
    
    # Vérifier le modèle SavedModel
    saved_model_path = os.path.join(deployed_path, "saved_model")
    if os.path.exists(saved_model_path):
        checks.append("✅ SavedModel présent")
        
        # Vérifier les fichiers critiques
        critical_files = ["saved_model.pb", "variables"]
        for file in critical_files:
            file_path = os.path.join(saved_model_path, file)
            if os.path.exists(file_path):
                checks.append(f"✅ {file} présent")
            else:
                checks.append(f"❌ {file} manquant")
    else:
        checks.append("❌ SavedModel manquant")
    
    # Vérifier les métadonnées
    metadata_path = os.path.join(deployed_path, "metadata.json")
    if os.path.exists(metadata_path):
        checks.append("✅ Métadonnées présentes")
        
        # Tester le chargement du JSON
        try:
            with open(metadata_path, 'r') as f:
                metadata = json.load(f)
            checks.append("✅ Métadonnées valides")
        except:
            checks.append("❌ Métadonnées corrompues")
    else:
        checks.append("❌ Métadonnées manquantes")
    
    # Vérifier la label map
    label_map_path = os.path.join(deployed_path, "label_map.pbtxt")
    if os.path.exists(label_map_path):
        checks.append("✅ Label map présente")
    else:
        checks.append("⚠️  Label map manquante (optionnelle)")
    
    # Test de chargement du modèle
    try:
        test_model = tf.saved_model.load(saved_model_path)
        checks.append("✅ Modèle chargeable")
        
        # Test d'inférence rapide
        if 'serving_default' in test_model.signatures:
            checks.append("✅ Signature d'inférence disponible")
        else:
            checks.append("⚠️  Signature 'serving_default' non trouvée")
            
    except Exception as e:
        checks.append(f"❌ Erreur de chargement: {str(e)[:50]}...")
    
    # Afficher les résultats
    for check in checks:
        print(f"   {check}")
    
    # Compter les succès
    success_count = sum(1 for check in checks if check.startswith("✅"))
    total_count = len([c for c in checks if not c.startswith("⚠️")])
    
    print(f"\n📊 Résultat: {success_count}/{total_count} vérifications réussies")
    
    if success_count >= total_count * 0.8:  # 80% de réussite minimum
        print("🎉 VALIDATION RÉUSSIE - Modèle prêt pour l'utilisation!")
        return True
    else:
        print("❌ VALIDATION ÉCHOUÉE - Vérifier les erreurs ci-dessus")
        return False

# Exécuter la validation si le déploiement a réussi
if 'deployed_path' in locals():
    validation_success = validate_deployment(deployed_path)
else:
    print("⚠️  Aucun déploiement à valider")


🔍 VALIDATION DU DÉPLOIEMENT
----------------------------------------
   ✅ Dossier de déploiement créé
   ✅ SavedModel présent
   ✅ saved_model.pb présent
   ✅ variables présent
   ✅ Métadonnées présentes
   ✅ Métadonnées valides
   ✅ Label map présente
   ✅ Modèle chargeable
   ✅ Signature d'inférence disponible

📊 Résultat: 9/9 vérifications réussies
🎉 VALIDATION RÉUSSIE - Modèle prêt pour l'utilisation!
